In [1]:
%pip install numpy==1.24.3
%pip install pandas==1.3.4
%pip install scikit-learn==0.23
%pip install tensorflow==2.13.0
%pip install matplotlib==3.5.0
%pip install pprint

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pprint (from versions: none)
ERROR: No matching distribution found for pprint


# Import the required libraries

In [2]:
import pandas as pd
import numpy as np
import keras
import sklearn

from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from pprint import pprint

# Download and Clean Dataset from https://cocl.us/concrete_data

In [3]:
import requests
r = requests.get('https://cocl.us/concrete_data')
with open('concrete_data.csv', 'wb') as f:
    f.write(r.content)

In [4]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

# check for missing values
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

# Split data into predictors and target

In [5]:
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

# Normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

In [6]:
predictors = (predictors - predictors.mean()) / predictors.std()
# target_norm = (target - target.mean()) / target.std()

# Use the Keras library to build a neural network with 3 hidden layers, each of 10 nodes and ReLU activation function. Use the adam optimizer and the mean squared error  as the loss function.



In [7]:
# Use the Keras library to build a neural network with three hidden layers, each of 10 nodes and ReLU activation function.. Use the adam optimizer and the mean squared error  as the loss function.
model = Sequential()
model.add(Dense(10, activation='relu', input_shape=(8,)))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                90        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Randomly split the data into a training and test sets by holding 30% of the data for testing. Use the train_test_split helper function from Scikit-learn.

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=1)

# Train the model on the training data using 50 epochs.

In [9]:
model.fit(X_train, y_train, epochs=50, verbose=2)

Epoch 1/50
23/23 - 1s - loss: 1536.2520 - 662ms/epoch - 29ms/step
Epoch 2/50
23/23 - 0s - loss: 1516.7883 - 19ms/epoch - 826us/step
Epoch 3/50
23/23 - 0s - loss: 1487.8416 - 19ms/epoch - 826us/step
Epoch 4/50
23/23 - 0s - loss: 1440.7382 - 19ms/epoch - 805us/step
Epoch 5/50
23/23 - 0s - loss: 1357.3538 - 19ms/epoch - 826us/step
Epoch 6/50
23/23 - 0s - loss: 1222.6984 - 21ms/epoch - 916us/step
Epoch 7/50
23/23 - 0s - loss: 1020.0544 - 18ms/epoch - 783us/step
Epoch 8/50
23/23 - 0s - loss: 763.3630 - 19ms/epoch - 826us/step
Epoch 9/50
23/23 - 0s - loss: 508.3513 - 32ms/epoch - 1ms/step
Epoch 10/50
23/23 - 0s - loss: 340.9871 - 32ms/epoch - 1ms/step
Epoch 11/50
23/23 - 0s - loss: 274.5055 - 33ms/epoch - 1ms/step
Epoch 12/50
23/23 - 0s - loss: 242.5791 - 29ms/epoch - 1ms/step
Epoch 13/50
23/23 - 0s - loss: 219.6234 - 21ms/epoch - 913us/step
Epoch 14/50
23/23 - 0s - loss: 205.0317 - 21ms/epoch - 918us/step
Epoch 15/50
23/23 - 0s - loss: 193.1330 - 20ms/epoch - 874us/step
Epoch 16/50
23/23 - 

# Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. Use the mean_squared_error function from Scikit-learn.

In [10]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print('Mean squared error:', mse)

10/10 [==============================] - 0s 778us/step
Mean squared error: 126.30516163808277


# Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

In [11]:
mse_list = []
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i+2)
    model.fit(X_train, y_train, epochs=50, verbose=0)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_list.append(mse)

10/10 [==============================] - 0s 890us/step


# Report the mean and the standard deviation of the mean squared errors.

In [12]:
print('Mean of mean squared errors:', np.mean(mse_list))
print('Standard deviation of mean squared errors:', np.std(mse_list))
pprint(mse_list)

Mean of mean squared errors: 26.219800966566694
Standard deviation of mean squared errors: 8.604348108449269
[61.6071662822773,
 51.943246170026256,
 42.83869388193538,
 39.30678218017656,
 46.67575058980777,
 30.998085195194356,
 34.0847164974794,
 30.086386829511977,
 30.21880290399695,
 23.82724875393723,
 29.733309708731145,
 31.33840327521273,
 26.711315467667276,
 20.848131330684247,
 24.371892113623396,
 25.509652333916733,
 24.286785054003506,
 28.447564691713662,
 25.689279829107445,
 26.117276618981528,
 19.981134161125887,
 24.730551589195965,
 23.910316736350588,
 27.17132102403807,
 22.751980825126893,
 22.87377097648771,
 27.183475699199818,
 25.72365054170694,
 20.5191749807741,
 18.731189792380928,
 19.724013152408283,
 21.002012104615574,
 22.719436833106084,
 23.82972860420105,
 18.73349950902132,
 20.785104817018542,
 22.002074412000454,
 20.079243180426086,
 21.41407534105047,
 21.029026888584276,
 21.837020766287992,
 22.102377869034665,
 24.805154554509034,
 19.71

# How does the mean of the mean squared errors compare to that from Step B?

In step B the mean of the mean squared error was 41.734891844177355. In this step the mean of the mean squared error equals 26.219800966566694.
The mean squared error is lower when compared to step A. The increased number of hidden layesr lowered the mean of the mean squared error.